In [1]:
import pandas as pd
import numpy as np

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
# plt.style.use('seaborn')             ggplot;
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'               #解决一些字体显示乱码问题
import warnings; warnings.simplefilter('ignore')       #忽略警告信息；

## 第一步，准备数据

In [3]:
#datapath=r'D:\Baiducloud\12M8-202005\1MIN\EURUSD_M1.csv'    #windows
datapath=r'/Users/nilei/Nutstore/Data/EURUSD_M1.csv'         #macos
dt=pd.read_csv(datapath)

In [4]:
dt.columns

Index(['Time (UTC)', 'Open', 'High', 'Low', 'Close', 'Volume '], dtype='object')

In [5]:
dt.rename(columns={'Time (UTC)':'Date'},inplace=True)
dt.index=pd.to_datetime(dt.Date)

In [8]:
dt.drop(columns={'Date','Volume '},inplace=True)

In [9]:
#截取需要的数据，分析一下2019-11-29至2019-12-31的一段上涨行情，其实是满 H4的金叉行情
data_1=dt['2019-11-29':'2020-1-1']
data_1.head()

,Open,High,Low,Close
Date,,,,
2019-11-29 00:00:00,1.10113,1.10113,1.10107,1.10112
2019-11-29 00:01:00,1.10112,1.10112,1.10107,1.10109
2019-11-29 00:02:00,1.10109,1.10109,1.10107,1.10107
2019-11-29 00:03:00,1.10107,1.10108,1.10107,1.10108
2019-11-29 00:04:00,1.10107,1.10108,1.10107,1.10108


In [10]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 32280 entries, 2019-11-29 00:00:00 to 2020-01-01 23:59:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    32280 non-null  float64
 1   High    32280 non-null  float64
 2   Low     32280 non-null  float64
 3   Close   32280 non-null  float64
dtypes: float64(4)
memory usage: 1.2 MB


In [11]:
data_1.describe()

,Open,High,Low,Close
count,32280.000000,32280.000000,32280.000000,32280.000000
mean,1.110771,1.110814,1.110728,1.110770
std,0.004884,0.004890,0.004878,0.004884
min,1.098200,1.098270,1.098110,1.098210
25%,1.107870,1.107910,1.107820,1.107870
50%,1.109520,1.109570,1.109470,1.109520
75%,1.113610,1.113650,1.113560,1.113610
max,1.123910,1.123910,1.123800,1.123890


##### 数据整理完毕

## 第二步，计算回测情况

In [ ]:
## np.cumsum()累计求和
## np.argmax()返回一个numpy数组中最大值的索引值
## np.argmin()返回一个numpy数组中最小值的索引值

In [13]:
data_1['return']=np.log(data_1['Close'] / data_1['Close'].shift(1))
data_1.head()

,Open,High,Low,Close,return
Date,,,,,
2019-11-29 00:00:00,1.10113,1.10113,1.10107,1.10112,NaN
2019-11-29 00:01:00,1.10112,1.10112,1.10107,1.10109,-0.000027
2019-11-29 00:02:00,1.10109,1.10109,1.10107,1.10107,-0.000018
2019-11-29 00:03:00,1.10107,1.10108,1.10107,1.10108,0.000009
2019-11-29 00:04:00,1.10107,1.10108,1.10107,1.10108,0.000000


In [14]:
data_1.fillna(0,inplace=True)

In [15]:
data_1.head()

,Open,High,Low,Close,return
Date,,,,,
2019-11-29 00:00:00,1.10113,1.10113,1.10107,1.10112,0.000000
2019-11-29 00:01:00,1.10112,1.10112,1.10107,1.10109,-0.000027
2019-11-29 00:02:00,1.10109,1.10109,1.10107,1.10107,-0.000018
2019-11-29 00:03:00,1.10107,1.10108,1.10107,1.10108,0.000009
2019-11-29 00:04:00,1.10107,1.10108,1.10107,1.10108,0.000000


In [16]:
#找到这段时间的最高点和最低点
data_1['cumsum']=data_1['return'].cumsum().apply(np.exp)

In [17]:
data_1.head()

,Open,High,Low,Close,return,cumsum
Date,,,,,,
2019-11-29 00:00:00,1.10113,1.10113,1.10107,1.10112,0.000000,1.000000
2019-11-29 00:01:00,1.10112,1.10112,1.10107,1.10109,-0.000027,0.999973
2019-11-29 00:02:00,1.10109,1.10109,1.10107,1.10107,-0.000018,0.999955
2019-11-29 00:03:00,1.10107,1.10108,1.10107,1.10108,0.000009,0.999964
2019-11-29 00:04:00,1.10107,1.10108,1.10107,1.10108,0.000000,0.999964


从哪些角度去思考，从而抓住特征  
1.空间，产生个几个高点；什么是高点，所谓的高点是一个一段时间内的price最大值；  
2.该最高点维持的时间有多久；  
3.跨度比较长的这段时间呢，价格经历了怎么样的曲折，又产生了怎么样的回撤；

#### 1.找出该时段内的最高点和最低点

In [18]:
cumsum_min=data_1['cumsum'].min()    #最小值是多少
round(cumsum_min,5)

0.99736

In [19]:
cumsum_max=data_1['cumsum'].max()   #最大值是多少
round(cumsum_max,5)

1.02068

In [20]:
amax=data_1['cumsum'].argmax()     #这个amax是列数
idmax=data_1['cumsum'].idxmax()
idmin=data_1['cumsum'].idxmin()
print(amax,idmax,idmin)

31682 2019-12-31 14:02:00 2019-11-29 13:14:00


In [21]:
a=data_1.loc[idmax,'cumsum'].round(4)
b=data_1.loc[idmin,'cumsum'].round(4)

In [22]:
data_1.iloc[amax,2].round(4)

1.1238

In [23]:
print('该段行情的最低点为{}，时间为{}'.format(b,idmin))
print('----------------------------------')
print('该段行情的最高点为{}，时间为{}'.format(a,idmax))
print('----------------------------------')
print('最大上涨幅度为{}个点'.format((a-b)*100000))

该段行情的最低点为0.9974，时间为2019-11-29 13:14:00
----------------------------------
该段行情的最高点为1.0207，时间为2019-12-31 14:02:00
----------------------------------
最大上涨幅度为2329.9999999999986个点


#### 2.利用rolling函数来，找出peak和valley

思考一下，怎么来定义peak和valley  
1.rolling只是固定时间段，能否考虑一下使用sma_cross来界定，因为这样的意义可能会更大一些，但是这样的话又会有sma周期的区分，这样问题的复杂度就会提高;  
2.在已经产生peak或者valley的前提下，是否需要衍生出peak_1 or valley_1;

In [ ]:
#设计一个函数来计算drawdown
def Drawdown(series):
    series=data_1['cumsum']
    drawdown=round((series.max()-series.min()),4)*100000
    return drawdown
Drawdown(data_1['cumsum'])